In [15]:
# ====================================================
# Resolvify - Corporate Complaint Management System
# Version: v3(Prime)
# Author: Rajdeep Debnath
# Environment: Google Colab
# ====================================================

import json
import datetime
import os
import re
import hashlib

# Optional nice output - attempt to import tabulate and termcolor
try:
    from tabulate import tabulate
except Exception:
    tabulate = None

try:
    from termcolor import colored
except Exception:
    colored = None

# -----------------------------
# Configuration / Globals
# -----------------------------
DATA_FILE = "complaints.json"
LOG_FILE = "log.txt"

# Admin password is "15201084" (we store only the hash)
ADMIN_HASH = hashlib.sha256("15201084".encode()).hexdigest()

DEPARTMENTS = [
    "IT Support",
    "HR Department",
    "Facilities",
    "Finance",
    "General"
]

# -----------------------------
# Utility Functions
# -----------------------------
def load_data():
    """Load complaint list from JSON file."""
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            return json.load(f)
    return []

def save_data(data):
    """Save complaint list to JSON file."""
    with open(DATA_FILE, "w") as f:
        json.dump(data, f, indent=4)

def log_action(action, user="System"):
    """Append timestamped action to log file."""
    with open(LOG_FILE, "a") as f:
        f.write(f"[{datetime.datetime.now()}] {user}: {action}\n")

def next_id(data):
    """Generate next sequential 3-digit ID (string)."""
    if not data:
        return "001"
    max_id = max(int(item["complaint_id"]) for item in data)
    return f"{max_id + 1:03d}"

def is_valid_email(email):
    """Basic regex email validation."""
    pattern = r'^[a-zA-Z0-9._%+\-]+@[a-zA-Z0-9.\-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def is_valid_name(name):
    """Name should not be empty or purely numeric."""
    return bool(name.strip()) and not name.strip().isdigit()

def print_colored(text, color=None):
    """Print colored text if termcolor available, else plain print."""
    if colored and color:
        print(colored(text, color))
    else:
        print(text)

def print_table(list_of_dicts):
    """Print a list of dicts as table if tabulate available, else simple print."""
    if not list_of_dicts:
        print_colored("No records to display.", "yellow")
        return
    if tabulate:
        # ensure consistent ordering of keys
        headers = list_of_dicts[0].keys()
        rows = [[d.get(h, "") for h in headers] for d in list_of_dicts]
        print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))
    else:
        # fallback
        for d in list_of_dicts:
            for k, v in d.items():
                print(f"{k}: {v}")
            print("-" * 30)

# -----------------------------
# User Functions (User Menu)
# -----------------------------
def choose_department():
    """Let user pick a department; returns string."""
    print_colored("\nSelect Department:", "cyan")
    for idx, dept in enumerate(DEPARTMENTS, start=1):
        print(f"{idx}. {dept}")
    choice = input("Choose (enter number): ").strip()
    if not choice.isdigit() or not (1 <= int(choice) <= len(DEPARTMENTS)):
        print_colored("Invalid selection. Defaulting to 'General'.", "yellow")
        return "General"
    return DEPARTMENTS[int(choice) - 1]

def register_complaint():
    """Register a new complaint (User)."""
    data = load_data()
    print_colored("\n--- Register a Complaint ---", "cyan")
    name = input("Enter your name: ").strip()
    if not is_valid_name(name):
        print_colored("❌ Invalid name. Registration cancelled.", "red")
        return
    email = input("Enter your email: ").strip()
    if not is_valid_email(email):
        print_colored("❌ Invalid email format. Registration cancelled.", "red")
        return
    department = choose_department()
    complaint_text = input("Enter your complaint description: ").strip()
    if not complaint_text:
        print_colored("❌ Complaint text cannot be empty. Registration cancelled.", "red")
        return

    complaint = {
        "complaint_id": next_id(data),
        "name": name,
        "email": email,
        "department": department,
        "complaint_text": complaint_text,
        "status": "Pending",
        "timestamp": str(datetime.datetime.now())
    }
    data.append(complaint)
    save_data(data)
    log_action(f"Registered complaint {complaint['complaint_id']} (Dept: {department})", name)
    print_colored(f"✅ Complaint Registered Successfully! Your Complaint ID: {complaint['complaint_id']}", "green")

def track_complaint_by_id():
    """Allow user to track complaint using complaint ID."""
    data = load_data()
    print_colored("\n--- Track Complaint by ID ---", "cyan")
    cid = input("Enter your Complaint ID (e.g., 001): ").strip()
    found = next((c for c in data if c["complaint_id"] == cid), None)
    if not found:
        print_colored("❌ Complaint ID not found.", "red")
        return
    print_colored("Complaint details:", "cyan")
    print_table([found])

def view_my_complaints():
    """User can view all complaints registered under their email."""
    data = load_data()
    print_colored("\n--- View My Complaints ---", "cyan")
    email = input("Enter your email: ").strip()
    if not is_valid_email(email):
        print_colored("❌ Invalid email format.", "red")
        return
    user_complaints = [c for c in data if c["email"].lower() == email.lower()]
    if not user_complaints:
        print_colored("No complaints found for this email.", "yellow")
        return
    print_colored(f"Complaints for {email}:", "cyan")
    print_table(user_complaints)

# -----------------------------
# Admin Functions (Admin Menu)
# -----------------------------
def admin_login():
    """Authenticate admin and show admin menu."""
    print_colored("\n--- Admin Login ---", "cyan")
    pwd = input("Enter Admin Password: ").strip()
    if hashlib.sha256(pwd.encode()).hexdigest() == ADMIN_HASH:
        print_colored("✅ Admin Login Successful.", "green")
        log_action("Admin logged in", "Admin")
        admin_menu()
    else:
        print_colored("❌ Incorrect password. Access denied.", "red")

def admin_menu():
    """Admin menu with all admin features."""
    while True:
        print_colored("\n=== Admin Panel ===", "magenta")
        print("1. View All Complaints")
        print("2. View Complaints by Department")
        print("3. Update Complaint Status")
        print("4. Delete Complaint by ID")
        print("5. Dashboard Summary (Dept & Status counts)")
        print("6. Search Complaints")
        print("7. Logout")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            view_all_complaints()
        elif choice == "2":
            view_by_department()
        elif choice == "3":
            update_status()
        elif choice == "4":
            delete_complaint()
        elif choice == "5":
            show_dashboard()
        elif choice == "6":
            search_complaints()
        elif choice == "7":
            print_colored("Logging out admin...", "yellow")
            break
        else:
            print_colored("❌ Invalid choice.", "red")

def view_all_complaints():
    """Show all complaints."""
    data = load_data()
    if not data:
        print_colored("No complaints registered yet.", "yellow")
        return
    print_colored("\nAll Registered Complaints:", "cyan")
    print_table(data)

def view_by_department():
    """Admin filter by department."""
    data = load_data()
    print_colored("\n--- View by Department ---", "cyan")
    for idx, dept in enumerate(DEPARTMENTS, start=1):
        print(f"{idx}. {dept}")
    choice = input("Choose department (enter number): ").strip()
    if not choice.isdigit() or not (1 <= int(choice) <= len(DEPARTMENTS)):
        print_colored("❌ Invalid department choice.", "red")
        return
    chosen = DEPARTMENTS[int(choice) - 1]
    filtered = [c for c in data if c.get("department") == chosen]
    if not filtered:
        print_colored(f"No complaints found for department '{chosen}'.", "yellow")
        return
    print_colored(f"\nComplaints for Department: {chosen}", "cyan")
    print_table(filtered)

def update_status():
    """Admin updates status by complaint ID."""
    data = load_data()
    print_colored("\n--- Update Complaint Status ---", "cyan")
    cid = input("Enter Complaint ID (e.g., 001): ").strip()
    for c in data:
        if c["complaint_id"] == cid:
            print_colored(f"Current status: {c['status']}", "yellow")
            new_status = input("Enter new status (Pending / In Progress / Resolved): ").strip().title()
            if new_status not in ["Pending", "In Progress", "Resolved"]:
                print_colored("❌ Invalid status. No change made.", "red")
                return
            c["status"] = new_status
            c["updated_on"] = str(datetime.datetime.now())
            save_data(data)
            log_action(f"Updated status for {cid} -> {new_status}", "Admin")
            print_colored(f"✅ Complaint {cid} status updated to '{new_status}'.", "green")
            return
    print_colored("❌ Complaint ID not found.", "red")

def delete_complaint():
    """Admin deletes complaint by ID."""
    data = load_data()
    print_colored("\n--- Delete Complaint ---", "cyan")
    cid = input("Enter Complaint ID to delete: ").strip()
    new_data = [c for c in data if c["complaint_id"] != cid]
    if len(new_data) == len(data):
        print_colored("❌ Complaint ID not found. Nothing deleted.", "red")
        return
    save_data(new_data)
    log_action(f"Deleted complaint {cid}", "Admin")
    print_colored(f"🗑️ Complaint {cid} deleted successfully.", "yellow")

def show_dashboard():
    """Show status-wise and department-wise counts."""
    data = load_data()
    if not data:
        print_colored("No data for dashboard.", "yellow")
        return

    # Status counts
    status_counts = {}
    for c in data:
        s = c.get("status", "Pending")
        status_counts[s] = status_counts.get(s, 0) + 1

    # Department counts
    dept_counts = {}
    for d in DEPARTMENTS:
        dept_counts[d] = sum(1 for c in data if c.get("department") == d)

    print_colored("\n=== Dashboard Summary ===", "cyan")
    # Print status counts
    print_colored("\nStatus-wise Counts:", "cyan")
    status_list = [{"Status": k, "Count": v} for k, v in status_counts.items()]
    print_table(status_list)

    # Print department counts
    print_colored("\nDepartment-wise Counts:", "cyan")
    dept_list = [{"Department": k, "Count": v} for k, v in dept_counts.items()]
    print_table(dept_list)

def search_complaints():
    """Search complaints by ID or keyword across fields."""
    data = load_data()
    print_colored("\n--- Search Complaints ---", "cyan")
    keyword = input("Enter Complaint ID or keyword: ").strip().lower()
    if not keyword:
        print_colored("❌ Empty search.", "red")
        return
    results = []
    for c in data:
        if (keyword == c["complaint_id"].lower() or
            keyword in c["complaint_text"].lower() or
            keyword in c["name"].lower() or
            keyword in c["email"].lower() or
            keyword in c.get("department", "").lower() or
            keyword in c.get("status", "").lower()):
            results.append(c)
    if not results:
        print_colored("❌ No matching complaints found.", "yellow")
        return
    print_colored(f"\nSearch Results ({len(results)}):", "cyan")
    print_table(results)

# -----------------------------
# Main Menu (Unified)
# -----------------------------
def main_menu():
    print_colored("\nWelcome to Resolvify - Corporate Complaint Management System (v3 Prime)", "magenta")
    while True:
        print_colored("\nMain Menu:", "cyan")
        print("1. Register Complaint (User)")
        print("2. Track Complaint by ID (User)")
        print("3. View My Complaints (User)")
        print("4. Admin Login")
        print("5. Exit")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            register_complaint()
        elif choice == "2":
            track_complaint_by_id()
        elif choice == "3":
            view_my_complaints()
        elif choice == "4":
            admin_login()
        elif choice == "5":
            print_colored("Exiting Resolvify. Goodbye!", "yellow")
            break
        else:
            print_colored("❌ Invalid option. Try again.", "red")

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    main_menu()


Welcome to Resolvify - Corporate Complaint Management System (v3 Prime)

Main Menu:
1. Register Complaint (User)
2. Track Complaint by ID (User)
3. View My Complaints (User)
4. Admin Login
5. Exit
Enter choice: 5
Exiting Resolvify. Goodbye!
